In [ ]:
!nvidia-smi

Sat Apr 17 11:32:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
## Mount grdive unit in order to load data and import source files
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
!ls /content/gdrive/MyDrive/ColabNotebooks/

'Copia de Project 2.ipynb'   microwave_out      redd.yaml
 dataset.py		     modelclassatt.py   run.sh
 dataset_test.py	     model.py	        settings.yaml
 dishwasher_out		    'Project 2.ipynb'   template.yaml
 fridge_out		    'Project antic'     utils.py
 __init__.py		     __pycache__
 main.py		     redd.py


In [ ]:
## Include ColabNotebooks to syspath to let python load libraries
import sys
sys.path.append('/content/gdrive/MyDrive/ColabNotebooks')

In [ ]:
!ls /content/gdrive/MyDrive/datasetKorea/redd

redd_house1_0.csv   redd_house1_8.csv  redd_house3_0.csv  redd_house4_3.csv
redd_house1_10.csv  redd_house1_9.csv  redd_house3_1.csv  redd_house4_4.csv
redd_house1_1.csv   redd_house2_0.csv  redd_house3_2.csv  redd_house4_5.csv
redd_house1_2.csv   redd_house2_1.csv  redd_house3_3.csv  redd_house5_0.csv
redd_house1_3.csv   redd_house2_2.csv  redd_house3_4.csv  redd_house6_0.csv
redd_house1_4.csv   redd_house2_3.csv  redd_house3_5.csv  redd_house6_1.csv
redd_house1_5.csv   redd_house2_4.csv  redd_house4_0.csv  redd_house6_2.csv
redd_house1_6.csv   redd_house2_5.csv  redd_house4_1.csv  redd_house6_3.csv
redd_house1_7.csv   redd_house2_6.csv  redd_house4_2.csv


In [ ]:
!pip install ray

In [ ]:
# -*- coding: utf-8 -*-
import os
import sys
from argparse import ArgumentParser

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from ray import tune

import matplotlib.pyplot as plt

from dataset import InMemoryKoreaDataset
from model import ModelPaperBackward as Model
from utils import error, load_yaml

####Single epoch (Train, Eval, Test)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def train_single_epoch(epoch, model, train_loader, optimizer, eval_loader,
    plotfilename=None):
    model.train()
    errs, losses = [], []
    x = torch.unsqueeze(x, dim=1)

    optimizer.zero_grad()
    x, y, clas = x.to(device), y.to(device), clas.to(device)

def eval_single_epoch(model, eval_loader, plotfilename=None):
    errs, losses = [], []
    with torch.no_grad():
        model.eval()
        for idx, (x, y, clas) in enumerate(eval_loader):
            x = torch.unsqueeze(x, dim=1)

            x, y, clas = x.to(device), y.to(device), clas.to(device)
            y_, reg_, alphas_, clas_ = model(x)

            loss_clas = F.binary_cross_entropy(clas_, clas)
            loss_out = F.mse_loss(y_, y)
            loss = loss_out + loss_clas
            err = error(y, y_)

            loss_, err_ = loss.item(), err.item()
            losses.append(loss_)
            errs.append(err_)

            if idx % 50 == 0:
              print(f"eval epoch={epoch} batch={idx+1} loss={loss:.2f} err={err:.2f}")
              if plotfilename:
                  filename = plotfilename + f".{idx}eval.png"
                  plot_window(
                      x.cpu(),
                      y.cpu(),
                      y_.cpu(),
                      reg_.cpu(),
                      clas_.cpu(),
                      alphas_.cpu(),
                      loss_,
                      err_,
                      filename,
                  )
    return np.mean(losses), np.mean(errs)


def test_single(model, test_loader, appliance, plotfilename=None):
    errs, losses = [], []
    with torch.no_grad():
        model.eval()
        for idx, (x, y, clas) in enumerate(test_loader):
            x = torch.unsqueeze(x, dim=1)

            x, y, clas = x.to(device), y.to(device), clas.to(device)
            y_, reg_, alphas_, clas_ = model(x)

            loss_clas = F.binary_cross_entropy(clas_, clas)
            loss_out = F.mse_loss(y_, y)
            loss = loss_out + loss_clas
            err = error(y, y_)

            loss_, err_ = loss.item(), err.item()
            losses.append(loss_)
            errs.append(err_)

            if idx % 500 == 0:
                print(f"eval batch={idx+1} loss={loss:.2f} err={err:.2f}")
                if plotfilename:
                    filename = plotfilename + f".{idx}.attention.png"
                    plot_window(
                        x.cpu(),
                        y.cpu(),
                        y_.cpu(),
                        reg_.cpu(),
                        clas_.cpu(),
                        alphas_.cpu(),
                        loss_,
                        err_,
                        filename,
                    )

    return np.mean(losses), np.mean(errs)


###Train model

In [ ]:
def train_model(datapath, output, appliance, hparams, doplot=None, load=True):
    buildings = appliance["buildings"]["train"]
    name = appliance["name"]
    params = appliance["hparams"]
    record_err = np.inf

    active_threshold = appliance.get("active_threshold", 0.15)
    active_ratio = appliance.get("active_ratio", 0.5)
    active_oversample = appliance.get("active_oversample", 2)

    my_dataset = InMemoryKoreaDataset(
        datapath,
        buildings,
        name,
        windowsize=params["L"],
        active_threshold=active_threshold,
        active_ratio=active_ratio,
        active_oversample=active_oversample,
    )

    total_size = len(my_dataset)
    train_size = int(hparams["train_size"] * (total_size))
    eval_size = total_size - train_size

    print("============= DATASET =============")
    print(f"Total size: {total_size}".format(total_size))
    print(f"Train size: {train_size}".format(train_size))
    print(f"Eval size: {eval_size}".format(eval_size))
    print("===================================")

    train_dataset, eval_dataset = torch.utils.data.random_split(
        my_dataset, (train_size, eval_size)
    )

    filename = os.path.join(output, "dataset.pt")
    save_dataset(train_dataset, eval_dataset, filename)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=hparams["batch_size"], shuffle=True
    )
    eval_loader = torch.utils.data.DataLoader(
        eval_dataset, batch_size=hparams["batch_size"]
    )

    model = Model(params["L"], params["F"], params["K"], params["H"])
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), hparams["lr"])
    # scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

    if load:
        filename = os.path.join(output, appliance["filename"])
        record_err = load_model(filename, model, optimizer)

    results = []
    for epoch in range(hparams["epochs"]):
        filename = os.path.join(output, appliance["filename"] + str(epoch))

        plotfilename = None
        if doplot:
            plotfilename = filename

        err_ = None
        try:
            train_single_epoch(
                epoch, model, train_loader, optimizer, eval_loader, plotfilename)

            loss_, err_ = eval_single_epoch(model, eval_loader)
            print("==========================================")
            print(f"eval loss={loss:.2f} err={err:.2f}")
            print("==========================================")

            # tune.report(eval_loss=loss_)
            results.append([(epoch, loss, err), (epoch, loss_, err_)])

            if err_ < record_err:
                filename = os.path.join(output, appliance["filename"])
                save(model, optimizer, filename, err_)
                record_err = err_
        except Exception as e:
            print(e)

        # scheduler.step()
    summary(output, results)

    return model


def train_model_wrapper(config):
    datapath = config["datapath"]
    output = config["output"]
    appliance = config["appliance"]
    hparams = config["hparams"]
    doplot = config["doplot"]
    load = config["load"]
    tune_hparams = config["tune"]

    appliance["hparams"]["F"] = tune_hparams["F"]
    appliance["hparams"]["K"] = tune_hparams["K"]
    appliance["hparams"]["H"] = tune_hparams["H"]

    return train_model(datapath, output, appliance, hparams, doplot, load)


###Test model

In [ ]:
def test_model(datapath, output, appliance, hparams, doplot=None):
    buildings = appliance["buildings"]["test"]
    name = appliance["name"]
    params = appliance["hparams"]

    filename = os.path.join(output, appliance["filename"])
    plotfilename = None
    if doplot:
        plotfilename = filename

    active_threshold = appliance.get("active_threshold", 0.15)
    active_ratio = appliance.get("active_ratio", 0.5)
    active_oversample = appliance.get("active_oversample", 2)

    my_dataset = InMemoryKoreaDataset(
        datapath,
        buildings,
        name,
        windowsize=params["L"],
        active_threshold=active_threshold,
        active_ratio=active_ratio,
        active_oversample=active_oversample,
    )

    my_dataset, _ = torch.utils.data.random_split(my_dataset, (len(my_dataset), 0))

    test_loader = torch.utils.data.DataLoader(
        my_dataset, batch_size=hparams["batch_size"]
    )

    model = Model(params["L"], params["F"], params["K"], params["H"])
    model = model.to(device)

    name = appliance["name"]
    filename = os.path.join(output, appliance["filename"])
    load_model(filename, model)

    output = os.path.join(output, f"{name}")
    loss, err = test_single(model, test_loader, appliance, plotfilename)
    print(f"Test loss={loss:.2f} err={err:.2f}")

###Save/load

In [ ]:
def save(model, optimizer, file_name_model, error):
    print("Guardant...")
    torch.save(
        {
            "error": error,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        },
        file_name_model,
    )
    print("Model guardat!")


def save_dataset(train_, test_, filename):
    torch.save({"train": train_, "test": test_}, filename)


def load_model(file_name_model, model, optimizer=None):
    print("Loading model...")
    if torch.cuda.is_available():
        state = torch.load(file_name_model)
    else:
        state = torch.load(file_name_model, map_location=torch.device("cpu"))

    model.load_state_dict(state["model_state_dict"])
    error = state["error"]
    print("Loaded model! Error rècord: {}".format(error))
    if optimizer:
        optimizer.load_state_dict(state["optimizer_state_dict"])
    return error

###Plot

In [ ]:
def plot(x, y, yhat, loss, err, filename):
    subplots = [221, 222, 223, 224]
    plt.figure(1, figsize=(10, 8))
    plt.subplots_adjust(top=0.88)
    for i in range(4):
        x_, y_, yhat_ = (
            x.detach().numpy()[i][0],
            y.detach().numpy()[i],
            yhat.detach().numpy()[i],
        )
        plt.subplot(subplots[i])
        plt.plot(range(len(x_)), x_, color="b", label="x")
        plt.plot(range(len(y_)), y_, color="g", label="y")
        plt.plot(range(len(yhat_)), yhat_, color="r", label="yhat")

    plt.suptitle(f"loss {loss:.2f} error {err:.2f}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(filename)
    plt.clf()


# def plot_window(x, y, yhat, alphas, loss, err, filename):
#    # Naive plot window
#    subplt_x = 4
#    subplt_y = 4
#    plt.figure(1, figsize=(20, 16))
#    plt.subplots_adjust(top=0.88)
#
#    idxs = np.random.randint(len(x), size=(subplt_x * subplt_y))
#    for i, idx in enumerate(idxs):
#        x_, y_, yhat_ = (
#            x.detach().numpy()[idx][0],
#            y.detach().numpy()[idx],
#            yhat.detach().numpy()[idx],
#        )
#        alphas_ = alphas.detach().numpy()[idx].flatten()
#        ax1 = plt.subplot(subplt_x, subplt_y, i + 1)
#        ax2 = ax1.twinx()
#        ax1.plot(range(len(x_)), x_, color="b", label="x")
#        ax1.plot(range(len(y_)), y_, color="r", label="y")
#        ax1.plot(range(len(yhat_)), yhat_, color="orange", label="yhat")
#        ax2.fill_between(
#            range(len(alphas_)), alphas_, alpha=0.5, color="lightgrey", label="alpha"
#        )
#
#    plt.suptitle(f"loss {loss:.2f} error {err:.2f}")
#    ax1.legend()
#    ax2.legend()
#    plt.legend()
#    plt.tight_layout()
#    plt.savefig(filename)
#    plt.clf()


def plot_window(x, y, yhat, reg, clas, alphas, loss, err, filename):
    subplt_x = 4
    subplt_y = 4
    plt.figure(1, figsize=(20, 16))
    plt.subplots_adjust(top=0.88)

    idxs = np.random.randint(len(x), size=(subplt_x * subplt_y))
    for i, idx in enumerate(idxs):
        x_, y_, yhat_, reg_, clas_ = (
            x.detach().numpy()[idx][0],
            y.detach().numpy()[idx],
            yhat.detach().numpy()[idx],
            reg.detach().numpy()[idx],
            clas.detach().numpy()[idx],
        )
        alphas_ = alphas.detach().numpy()[idx].flatten()
        ax1 = plt.subplot(subplt_x, subplt_y, i + 1)
        ax2 = ax1.twinx()
        ax1.plot(range(len(x_)), x_, color="b", label="x")
        ax1.plot(range(len(y_)), y_, color="r", label="y")
        ax1.plot(range(len(reg_)), reg_, color="black", label="reg")
        ax1.plot(range(len(yhat_)), yhat_, alpha=0.5, color="orange", label="yhat")
        ax2.fill_between(
            range(len(alphas_)), alphas_, alpha=0.5, color="lightgrey", label="alpha"
        )
        alphas_max = np.max(alphas_)
        ax2.plot(
            range(len(clas_)), clas_ * alphas_max, color="cyan", alpha=0.25, label="reg"
        )

    plt.suptitle(f"loss {loss:.2f} error {err:.2f}")
    ax1.legend()
    ax2.legend()
    plt.legend()
    plt.tight_layout()
    plt.savefig(filename)
    plt.clf()


def summary(path, results):
    df = pd.DataFrame(
        [
            {
                "epoch": x[0][0],
                "train_loss": x[0][1],
                "train_err": x[0][2],
                "eval_loss": x[1][1],
                "eval_err": x[1][2],
            }
            for x in results
        ]
    ).set_index("epoch")

    columns = ["train_loss", "eval_loss"]
    filename = os.path.join(path, "results-loss.csv")
    df[columns].round(3).to_csv(filename, sep=";")
    filename = os.path.join(path, "results-loss.png")

    plt.figure(1, figsize=(10, 8))
    df[columns].round(3).plot()
    plt.savefig(filename)
    plt.clf()

    columns = ["train_err", "eval_err"]
    filename = os.path.join(path, "results-error.csv")
    df[columns].round(3).to_csv(filename, sep=";")
    filename = os.path.join(path, "results-error.png")

    plt.figure(1, figsize=(10, 8))
    df[columns].round(3).plot()
    plt.savefig(filename)
    plt.clf()

###Main

In [ ]:
def main(args):

    if args["disable_random"]:
        torch.manual_seed(7)

    train = args["train"]
    tune_enabled = args["tune"]
    output = args["path"]
    plot_disabled = args["disable_plot"]

    settings = load_yaml(args["settings"])
    appliance = args["appliance"]

    dataset = settings["dataset"]
    hparams = settings["hparams"]
    if args["epochs"]:
        hparams["epochs"] = int(args["epochs"])

    appliance = settings["appliances"][appliance]

    datapath = dataset["path"]
    if train:
        print("==========================================")
        print(f"Training ONGOING")
        print("==========================================")

        if not tune_enabled:
            my_model = train_model(
                datapath,
                output,
                appliance,
                hparams,
                doplot=not plot_disabled,
                load=True
            )
        else:
            config = {
                "datapath": datapath,
                "output": output,
                "appliance": appliance,
                "hparams": hparams,
                "doplot": not plot_disabled,
                "load": False,
                "tune": {
                    "F": tune.grid_search([16, 32, 64]),
                    "K": tune.grid_search([4, 8, 16]),
                    "H": tune.grid_search([256, 512, 1024]),
                },
            }
            analysis = tune.run(
                train_model_wrapper,
                metric="val_loss",
                mode="min",
                num_samples=5,
                config=config,
            )
            print("==========================================")
            print(f"Best hyperparameters")
            print(analysis.best_config)
            print("==========================================")

        print("==========================================")
        print(f"Training DONE")
        print("==========================================")
    else:
        print("==========================================")
        print(f"Testing ONGOING")
        print("==========================================")
        test_model(datapath, output, appliance, hparams, doplot=not plot_disabled)
        print("==========================================")
        print(f"Testing DONE")
        print("==========================================")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Args i execució

In [ ]:
args = {"settings": "/content/gdrive/MyDrive/ColabNotebooks/settings.yaml", "appliance": "microwave",
        "path": "/content/gdrive/MyDrive/ColabNotebooks/microwave_out", "train": True, "epochs": 1,
        "disable_random": True, "tune": False, "disable_plot": False}

main(args)